In [54]:
import pandas as pd
loans_2007 = pd.read_csv('LoanStats3a.csv', skiprows=1)
half_count = len(loans_2007) / 2

# Remove columns containing more than 50% null values
loans_2007 = loans_2007.dropna(thresh=half_count, axis=1)

# Remove text explanations and links for the loans
loans_2007 = loans_2007.drop(['desc', 'url'],axis=1)

/home/mark/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [55]:
print(loans_2007.iloc[0])
print(loans_2007.shape[1])

id                                1077501
member_id                      1.2966e+06
loan_amnt                            5000
funded_amnt                          5000
funded_amnt_inv                      4975
term                            36 months
int_rate                           10.65%
installment                        162.87
grade                                   B
sub_grade                              B2
emp_title                             NaN
emp_length                      10+ years
home_ownership                       RENT
annual_inc                          24000
verification_status              Verified
issue_d                          Dec-2011
loan_status                    Fully Paid
pymnt_plan                              n
purpose                       credit_card
title                            Computer
zip_code                            860xx
addr_state                             AZ
dti                                 27.65
delinq_2yrs                       

First, we want to identify features which exhibit the following:

* leak information from the future (after the loan has already been funded)
* don't affect a borrower's ability to pay back a loan (e.g. a randomly generated ID value by Lending Club)
* formatted poorly and need to be cleaned up
* require more data or a lot of processing to turn into a useful feature
* contain redundant information


Columns that need to be removed
* `id` : randomly generated field by Lending Club for unique identification purposes only
* `member_id` : also a randomly generated field by Lending Club for unique identification purposes only
* `funded_amnt`: leaks data from the future (after the loan is already started to be funded)
* `funded_amnt_inv` : also leaks data from the future (after the loan is already started to be funded)
* `grade` : contains redundant information as the interest rate column (int_rate)
* `sub_grade` : also contains redundant information as the interest rate column (int_rate)
* `emp_title` : requires other data and a lot of processing to potentially be useful
* `issue_d` : leaks data from the future (after the loan is already completed funded)


In [56]:
loans_2007.drop(['id', 'member_id', 'funded_amnt', 'funded_amnt_inv', 'grade', 'sub_grade', 'emp_title', 'issue_d'], axis=1, inplace=True)
print(loans_2007.iloc[0])

loan_amnt                            5000
term                            36 months
int_rate                           10.65%
installment                        162.87
emp_length                      10+ years
home_ownership                       RENT
annual_inc                          24000
verification_status              Verified
loan_status                    Fully Paid
pymnt_plan                              n
purpose                       credit_card
title                            Computer
zip_code                            860xx
addr_state                             AZ
dti                                 27.65
delinq_2yrs                             0
earliest_cr_line                 Jan-1985
inq_last_6mths                          1
open_acc                                3
pub_rec                                 0
revol_bal                           13648
revol_util                          83.7%
total_acc                               9
initial_list_status               

Also need to be removed:

* `zip_code` : redundant with the addr_state column since only the first 3 digits of the 5 digit zip code are visible (which only can be used to identify the state the borrower lives in)
* `out_prncp` : leaks data from the future, (after the loan already started to be paid off)
* `out_prncp_inv` : also leaks data from the future, (after the loan already started to be paid off)
* `total_pymnt` : also leaks data from the future, (after the loan already started to be paid off)
* `total_pymnt_inv` : also leaks data from the future, (after the loan already started to be paid off)
* `total_rec_prncp` : also leaks data from the future, (after the loan already started to be paid off)

In [57]:
loans_2007.drop(['zip_code', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp'], axis=1, inplace=True)
print(loans_2007.iloc[0])

loan_amnt                            5000
term                            36 months
int_rate                           10.65%
installment                        162.87
emp_length                      10+ years
home_ownership                       RENT
annual_inc                          24000
verification_status              Verified
loan_status                    Fully Paid
pymnt_plan                              n
purpose                       credit_card
title                            Computer
addr_state                             AZ
dti                                 27.65
delinq_2yrs                             0
earliest_cr_line                 Jan-1985
inq_last_6mths                          1
open_acc                                3
pub_rec                                 0
revol_bal                           13648
revol_util                          83.7%
total_acc                               9
initial_list_status                     f
total_rec_int                     

Also need to be removed:

* `total_rec_int` : leaks data from the future, (after the loan already started to be paid off),
* `total_rec_late_fee` : also leaks data from the future, (after the loan already started to be paid off),
* `recoveries` : also leaks data from the future, (after the loan already started to be paid off),
* `collection_recovery_fee` : also leaks data from the future, (after the loan already started to be paid off),
* `last_pymnt_d` : also leaks data from the future, (after the loan already started to be paid off),
* `last_pymnt_amnt`: also leaks data from the future, (after the loan already started to be paid off).


In [58]:
loans_2007.drop(['total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt'], axis=1, inplace=True)
print(loans_2007.iloc[0])

loan_amnt                            5000
term                            36 months
int_rate                           10.65%
installment                        162.87
emp_length                      10+ years
home_ownership                       RENT
annual_inc                          24000
verification_status              Verified
loan_status                    Fully Paid
pymnt_plan                              n
purpose                       credit_card
title                            Computer
addr_state                             AZ
dti                                 27.65
delinq_2yrs                             0
earliest_cr_line                 Jan-1985
inq_last_6mths                          1
open_acc                                3
pub_rec                                 0
revol_bal                           13648
revol_util                          83.7%
total_acc                               9
initial_list_status                     f
last_credit_pull_d               F

In [59]:
print(loans_2007.loan_status.value_counts())

Fully Paid                                             34115
Charged Off                                             5668
Does not meet the credit policy. Status:Fully Paid      1988
Does not meet the credit policy. Status:Charged Off      761
Late (31-120 days)                                         3
Name: loan_status, dtype: int64


## Clean target column

In [60]:
fully_paid = loans_2007.loan_status == "Fully Paid"
charged_off = loans_2007.loan_status == "Charged Off"
print(fully_paid.shape)
print(charged_off.shape)
both = fully_paid | charged_off
loans_2007 = loans_2007[both]
print(loans_2007.loan_status.value_counts())

mapping_dict = {
    'loan_status': {
        'Fully Paid' : 1,
        'Charged Off' : 0
    }
}

loans_2007 = loans_2007.replace(mapping_dict)
print(loans_2007.loan_status.value_counts())

(42538,)
(42538,)
Fully Paid     34115
Charged Off     5668
Name: loan_status, dtype: int64
1    34115
0     5668
Name: loan_status, dtype: int64


## Drop columns with only one unique value

In [61]:
drop_columns = []
for c in loans_2007.columns:
    loans_2007[c].dropna(inplace=True)
    if len(loans_2007[c].unique()) == 1:
        drop_columns.append(c)

loans_2007.drop(drop_columns, axis=1, inplace=True)
print(drop_columns)

['pymnt_plan', 'initial_list_status', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt', 'tax_liens']


In [62]:
null_counts = loans_2007.isnull().sum()
print(null_counts)

loan_amnt                 0
term                      0
int_rate                  0
installment               0
emp_length                0
home_ownership            0
annual_inc                0
verification_status       0
loan_status               0
purpose                   0
title                    10
addr_state                0
dti                       0
delinq_2yrs               0
earliest_cr_line          0
inq_last_6mths            0
open_acc                  0
pub_rec                   0
revol_bal                 0
revol_util               50
total_acc                 0
last_credit_pull_d        2
pub_rec_bankruptcies    697
dtype: int64


### Drop column containing more than 1% null values and rows containing null values

In [63]:
loans_2007.drop('pub_rec_bankruptcies', axis=1, inplace=True)
loans_2007.dropna(inplace=True)

print(loans_2007.dtypes.value_counts())

object     11
float64    10
int64       1
dtype: int64


In [64]:
object_columns_df = loans_2007.select_dtypes(include=['object'])
print(object_columns_df.iloc[0])

term                     36 months
int_rate                    10.65%
emp_length               10+ years
home_ownership                RENT
verification_status       Verified
purpose                credit_card
title                     Computer
addr_state                      AZ
earliest_cr_line          Jan-1985
revol_util                   83.7%
last_credit_pull_d        Feb-2017
Name: 0, dtype: object


### Evaluate features which may be discrete sets

In [65]:
cols = ['home_ownership', 'verification_status', 'emp_length', 'term', 'addr_state']

for c in cols:
    print(c, ": ")
    print(loans_2007[c].value_counts())
    print("\n")

home_ownership : 
RENT        18880
MORTGAGE    17687
OWN          3055
OTHER          96
NONE            3
Name: home_ownership, dtype: int64


verification_status : 
Not Verified       16890
Verified           12832
Source Verified     9999
Name: verification_status, dtype: int64


emp_length : 
10+ years    8896
< 1 year     4576
2 years      4389
3 years      4093
4 years      3435
5 years      3279
1 year       3240
6 years      2227
7 years      1771
8 years      1482
9 years      1259
n/a          1074
Name: emp_length, dtype: int64


term : 
 36 months    29041
 60 months    10680
Name: term, dtype: int64


addr_state : 
CA    7095
NY    3815
FL    2868
TX    2728
NJ    1850
IL    1524
PA    1515
VA    1407
GA    1399
MA    1343
OH    1221
MD    1053
AZ     878
WA     841
CO     791
NC     788
CT     754
MI     722
MO     685
MN     613
NV     497
SC     472
WI     459
AL     450
OR     450
LA     436
KY     327
OK     299
KS     271
UT     259
AR     245
DC     212
RI     199


In [66]:
print(loans_2007.purpose.value_counts())
print(loans_2007.title.value_counts())

debt_consolidation    18659
credit_card            5134
other                  3984
home_improvement       2980
major_purchase         2182
small_business         1827
car                    1549
wedding                 947
medical                 693
moving                  581
house                   382
vacation                380
educational             320
renewable_energy        103
Name: purpose, dtype: int64
Debt Consolidation                                   2188
Debt Consolidation Loan                              1731
Personal Loan                                         661
Consolidation                                         516
debt consolidation                                    508
Credit Card Consolidation                             357
Home Improvement                                      357
Debt consolidation                                    334
Small Business Loan                                   329
Credit Card Loan                                      319


In [67]:
loans_2007.drop(['last_credit_pull_d', 'addr_state', 'title', 'earliest_cr_line'], axis=1, inplace=True) 

In [68]:
mapping_dict = {
    "emp_length": {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        "n/a": 0
    }
}

loans_2007.int_rate = loans_2007['int_rate'].str.rstrip('%').astype(float)
loans_2007.revol_util = loans_2007.revol_util.str.rstrip('%').astype(float)
loans_2007.replace(mapping_dict, inplace=True)

In [69]:
loans_2007.head()

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc
0,5000.0,36 months,10.65,162.87,10,RENT,24000.0,Verified,1,credit_card,27.65,0.0,1.0,3.0,0.0,13648.0,83.7,9.0
1,2500.0,60 months,15.27,59.83,0,RENT,30000.0,Source Verified,0,car,1.00,0.0,5.0,3.0,0.0,1687.0,9.4,4.0
2,2400.0,36 months,15.96,84.33,10,RENT,12252.0,Not Verified,1,small_business,8.72,0.0,2.0,2.0,0.0,2956.0,98.5,10.0
3,10000.0,36 months,13.49,339.31,10,RENT,49200.0,Source Verified,1,other,20.00,0.0,1.0,10.0,0.0,5598.0,21.0,37.0
4,3000.0,60 months,12.69,67.79,1,RENT,80000.0,Source Verified,1,other,17.94,0.0,0.0,15.0,0.0,27783.0,53.9,38.0


In [78]:
# Returns a new Dataframe containing 1 column for each dummy variable.
dummies = ['home_ownership', 'verification_status', 'purpose', 'term']

for d in dummies:
    loans_2007[d] = loans_2007[d].astype('category')
    
dummy_df = pd.get_dummies(loans_2007[dummies])
loans_2007 = pd.concat([loans_2007, dummy_df], axis=1)
loans_2007.drop(dummies, axis=1, inplace=True)

loans_2007.head()

,loan_amnt,int_rate,installment,emp_length,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,...,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,term_ 36 months,term_ 60 months
0,5000.0,10.65,162.87,10,24000.0,1,27.65,0.0,1.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2500.0,15.27,59.83,0,30000.0,0,1.00,0.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2400.0,15.96,84.33,10,12252.0,1,8.72,0.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,10000.0,13.49,339.31,10,49200.0,1,20.00,0.0,1.0,10.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,3000.0,12.69,67.79,1,80000.0,1,17.94,0.0,0.0,15.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


### Fit a logistic regression and make predictions

In [81]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

features = loans_2007.drop('loan_status', axis=1)
target = loans_2007.loan_status

lr.fit(features, target)
predictions = lr.predict(features)

In [87]:
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_predict, KFold

lr = LogisticRegression()
kf = KFold(features.shape[0], random_state=1)
predictions = cross_val_predict(lr, features, target, cv=kf)
predictions = pd.Series(predictions)

tn = len(predictions[(predictions == 0) & (target == 0)])
tp = len(predictions[(predictions == 1) & (target == 1)])
fn = len(predictions[(predictions == 0) & (target == 1)])
fp = len(predictions[(predictions == 1) & (target == 0)])

fpr = fp / (fp + tn)
tpr = tp / (tp + fn)
print("False positive rate:", fpr)
print("True positive rate:", tpr)

False positive rate: 0.9989363588016309
True positive rate: 0.9991768337498163


### Ways to account for target imbalance
* Use oversampling and undersampling to ensure that the classifier gets input that has a balanced number of each class.
* Tell the classifier to penalize misclassifications of the less prevalent class more than the other class.

In [89]:
# This tells scikit-learn to penalize the misclassification of the minority class during the training process. 
# The penalty means that the logistic regression classifier pays more attention to correctly classifying rows 
# where loan_status is 0. 
# This lowers accuracy when loan_status is 1, but raises accuracy when loan_status is 0.
lr = LogisticRegression(class_weight='balanced')

kf = KFold(features.shape[0], random_state=1)
predictions = cross_val_predict(lr, features, target, cv=kf)
predictions = pd.Series(predictions)

tn = len(predictions[(predictions == 0) & (target == 0)])
tp = len(predictions[(predictions == 1) & (target == 1)])
fn = len(predictions[(predictions == 0) & (target == 1)])
fp = len(predictions[(predictions == 1) & (target == 0)])

fpr = fp / (fp + tn)
tpr = tp / (tp + fn)
print("False positive rate:", fpr)
print("True positive rate:", tpr)

False positive rate: 0.6041482006736394
True positive rate: 0.6218139056298692


### Manually penalize false positives 

In [94]:
penalty = {
    0: 10,
    1: 1
}

lr = LogisticRegression(class_weight=penalty)

kf = KFold(features.shape[0], random_state=1)
predictions = cross_val_predict(lr, features, target, cv=kf)
predictions = pd.Series(predictions)

tn = len(predictions[(predictions == 0) & (target == 0)])
tp = len(predictions[(predictions == 1) & (target == 1)])
fn = len(predictions[(predictions == 0) & (target == 1)])
fp = len(predictions[(predictions == 1) & (target == 0)])

fpr = fp / (fp + tn)
tpr = tp / (tp + fn)
print("False positive rate:", fpr)
print("True positive rate:", tpr)

False positive rate: 0.1962418010990959
True positive rate: 0.19870645303542556


### Random forests

In [101]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(class_weight='balanced', random_state=1)

kf = KFold(features.shape[0], random_state=1)
predictions = cross_val_predict(clf, features, target, cv=kf)
predictions = pd.Series(predictions)

tn = len(predictions[(predictions == 0) & (target == 0)])
tp = len(predictions[(predictions == 1) & (target == 1)])
fn = len(predictions[(predictions == 0) & (target == 1)])
fp = len(predictions[(predictions == 1) & (target == 0)])

fpr = fp / (fp + tn)
tpr = tp / (tp + fn)
print("False positive rate:", fpr)
print("True positive rate:", tpr)

False positive rate: 0.9693316787803581
True positive rate: 0.9709539908863737


### Other things to try
* We can tweak the penalties further.
* We can try models other than a random forest and logistic regression.
* We can use some of the columns we discarded to generate better features.
* We can ensemble multiple models to get more accurate predictions.
* We can tune the parameters of the algorithm to achieve higher performance.